<a href="https://colab.research.google.com/github/amilyk/HappyAgents/blob/main/Camel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install camel-ai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 55.1 MB/s eta 0:00:00
  Attempting uninstall: psutil
    Found existing installation: psutil 5.9.5
    Uninstalling psutil-5.9.5:
      Successfully uninstalled psutil-5.9.5
  Attempting uninstall: pillow
    Found existing installation: pillow 11.3.0
    Uninstalling pillow-11.3.0:
      Successfully uninstalled pillow-11.3.0
  Attempting uninstall: tiktoken
    Found existing installation: tiktoken 0.12.0
    Uninstalling tiktoken-0.12.0:
      Successfully uninstalled tiktoken-0.12.0


In [2]:
!pip install colorama

# 模型改成zhipu

In [1]:
# !touch .env
# !echo "ZHIPUAI_API_KEY='zhipu-api-key'" > .env

In [59]:
from colorama import init as colorama_init
colorama_init(autoreset=True)

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [3]:
from camel.models import ModelFactory
from camel.types import ModelPlatformType, ModelType

zhipu_key = os.getenv("ZHIPUAI_API_KEY")
zhipu_url = "https://open.bigmodel.cn/api/paas/v4"
# RolePlaying（以及底层的 ChatSession）在调用模型时会 自行拼接{self.base_url}/chat/completions，仅写到v4即可

model = ModelFactory.create(
    model_platform=ModelPlatformType.ZHIPU,
    model_type=ModelType.GLM_4,
    api_key=zhipu_key,
    url=zhipu_url,
    model_config_dict={                             # 所有超参数都放这里
        "temperature": 0.7,
        "max_tokens": 2048
    },
)

# 定义协作任务

In [7]:
from colorama import Fore
from camel.societies import RolePlaying
from camel.utils import print_text_animated

# 定义协作任务
task_prompt = """
创作一本关于"拖延症心理学"的短篇电子书，目标读者是对心理学感兴趣的普通大众。请始终使用简体中文回复。
要求：
1. 内容科学严谨，基于实证研究
2. 语言通俗易懂，避免过多专业术语
3. 包含实用的改善建议和案例分析
4. 篇幅控制在8000-10000字
5. 结构清晰，包含引言、核心章节和总结
"""

print(Fore.YELLOW + f"协作任务:\n{task_prompt}\n")

协作任务:

创作一本关于"拖延症心理学"的短篇电子书，目标读者是对心理学感兴趣的普通大众。请始终使用简体中文回复。
要求：
1. 内容科学严谨，基于实证研究
2. 语言通俗易懂，避免过多专业术语
3. 包含实用的改善建议和案例分析
4. 篇幅控制在8000-10000字
5. 结构清晰，包含引言、核心章节和总结




# 角色定义

In [8]:
# 初始化角色扮演会话

# AI 作家作为 "user"，负责提出写作结构和要求
# AI 心理学家作为 "assistant"，负责提供专业知识和内容
role_play_session = RolePlaying(
    model = model,
    assistant_role_name="心理学家",
    user_role_name="作家",
    task_prompt=task_prompt,
    with_task_specify=False, # 在本例中，我们直接使用给定的task_prompt
)

# # 关键：覆盖系统提示，使两边都使用中文
# role_play_session.assistant_sys_msg = "你是心理学家，请始终使用简体中文回复。"
# role_play_session.user_sys_msg      = "你是作家，请用中文提出需求。"


print(Fore.CYAN + f"具体任务描述:\n{role_play_session.task_prompt}\n")
# 两个角色的名称和任务

具体任务描述:

创作一本关于"拖延症心理学"的短篇电子书，目标读者是对心理学感兴趣的普通大众。请始终使用简体中文回复。
要求：
1. 内容科学严谨，基于实证研究
2. 语言通俗易懂，避免过多专业术语
3. 包含实用的改善建议和案例分析
4. 篇幅控制在8000-10000字
5. 结构清晰，包含引言、核心章节和总结




# 启动运行自动化对话

In [9]:
# 开始协作对话
# 一轮完整的交互（作家提需求、心理学家给内容），并将上一轮心理学家的输出作为下一轮的输入
chat_turn_limit, n = 30, 0
# 调用 init_chat() 来获得由 AI 生成的初始对话消息
input_msg = role_play_session.init_chat()

while n < chat_turn_limit:
    n += 1
    # step() 方法驱动一轮完整的对话，AI 用户和 AI 助理各发言一次
    assistant_response, user_response = role_play_session.step(input_msg)

    # 检查是否有消息返回，防止对话提前终止
    if assistant_response.msg is None or user_response.msg is None:
        break

    print_text_animated(Fore.BLUE + f"作家：\n\n{user_response.msg.content}\n")
    print_text_animated(Fore.GREEN + f"心理学家：\n\n{assistant_response.msg.content}\n")

    # 检查任务完成标志
    if "<CAMEL_TASK_DONE>" in user_response.msg.content or "<CAMEL_TASK_DONE>" in assistant_response.msg.content:
        print(Fore.MAGENTA + "✅ 电子书创作完成！")
        break

    # 将助理的回复作为下一轮对话的输入
    input_msg = assistant_response.msg

print(Fore.YELLOW + f"总共进行了 {n} 轮协作对话")


作家：

Instruction: 提供一个关于拖延症心理学的电子书大纲
Input: 大纲应包含引言、核心章节（至少三个）和总结，每个章节聚焦一个拖延症相关的心理主题。
心理学家：

Solution: 

以下是关于"拖延症心理学"的电子书大纲建议：

### 引言
- 简要介绍拖延症现象的普遍性及其对个人生活、工作的影响。
- 阐述本书目的：帮助读者理解拖延症的心理机制，并提供实用的改善建议。

### 核心章节

#### 第一章：拖延症的心理根源
- 主题：探讨拖延症的心理学基础，如完美主义、自我效能感、动机等。
  - 1.1 完美主义与拖延
    - 描述完美主义者的心态特点及其如何导致拖延。
    - 案例分析：一个完美主义者的拖延故事。
  - 1.2 自我效能感与拖延
    - 解释自我效能感对拖延的影响。
    - 实证研究：分享提高自我效能感以减少拖延的研究成果。
- 实用建议：如何调整心态，降低完美主义带来的负面影响。

#### 第二章：拖延症的认知行为疗法
- 主题：介绍认知行为疗法（CBT）在应对拖延症中的应用。
  - 2.1 认知行为疗法的原理
    - 简要介绍CBT的基本概念和原理。
  - 2.2 拖延症的认知误区
    - 分析拖延者常见的认知误区，如过度乐观估计、全或无思维等。
  - 2.3 CBT在拖延症治疗中的应用
    - 详细的案例分析和实用技巧。
- 实用建议：如何运用CBT技巧识别和改变拖延思维。

#### 第三章：拖延症的应对策略与时间管理
- 主题：探讨如何通过时间管理和制定应对策略来克服拖延。
  - 3.1 时间管理与拖延
    - 分析时间管理技巧在克服拖延中的重要性。
    - 实用时间管理工具介绍。
  - 3.2 制定应对策略
    - 分享制定明确目标、分解任务、设定截止日期等策略。
  - 3.3 应对拖延的心理学技巧
    - 如正念冥想、自我激励等。
- 实用建议：如何制定个人化的应对拖延策略。

### 总结
- 回顾全书主要观点和实用建议。
- 强调改变拖延行为需要时间和持续的努力，鼓励读者实践所学。

Next request.
作家：

Instruction: 针对第一章的内容，提供一些具体的实证研究案例，以便在书中引用。
Input: 案例应涉及完

作家：

Instruction: 针对第二章，提供认知行为疗法（CBT）中用于治疗拖延症的具体技术和策略。
Input: 这些技术和策略需要结合实际案例进行说明，以便读者更好地理解和应用。
心理学家：

Solution:

#### 第二章：拖延症的认知行为疗法

##### CBT中用于治疗拖延症的具体技术和策略

- **技术和策略一：认知重构**
  - **案例说明：**
    一位拖延者经常推迟完成工作报告，因为她总是担心自己的工作不够完美，害怕遭到同事的批评。在CBT治疗中，她学会了识别这种灾难化思维（认为如果报告不完美，就会遭受严厉批评），并通过认知重构技术替换为更加现实和积极的思维模式。例如，她学会了告诉自己：“我的报告不需要完美，只要尽力而为，同事们会理解并给予建设性反馈。”
  - **具体技术：**
    1. 识别和挑战负面核心信念。
    2. 使用逻辑和证据来对抗灾难化思维。
    3. 替换负面思维为更加现实和积极的思维。

- **技术和策略二：行为激活**
  - **案例说明：**
    一位拖延者发现自己很难开始任何项目，因为任务似乎总是太大、太复杂。在CBT治疗中，他学习了行为激活策略，将大任务分解为小步骤，并设定了具体、可实现的目标。例如，他开始每天只专注于完成项目的一小部分，而不是试图一次完成整个项目。
  - **具体技术：**
    1. 分解任务为小步骤，逐步完成任务。
    2. 设定具体、短期和可实现的目标。
    3. 使用行为契约或奖励系统来增加任务的积极性和即时性。

- **技术和策略三：正念和自我觉察**
  - **案例说明：**
    一位拖延者意识到她在面对困难任务时会感到焦虑，而这种焦虑会导致她逃避任务。通过CBT中的正念练习，她学会了如何观察自己的情绪和冲动，而不是盲目地跟随它们。她学会了在焦虑时停下来，进行深呼吸和自我觉察练习，这有助于她更好地管理情绪并重新聚焦于任务。
  - **具体技术：**
    1. 正念冥想，专注于当前的感受和体验。
    2. 自我觉察练习，观察自己的情绪和行为模式。
    3. 接受自己的情绪，而不是试图避免或抑制它们。

通过这些案例和具体技术的说明，读者可以更好地理解CBT是如何帮助人们识别和改变导致拖延的认知和行为模式。这些策

ERROR:camel.models.model_manager:Error processing with model: <camel.models.zhipuai_model.ZhipuAIModel object at 0x7db1563c9550>
ERROR:camel.models.model_manager:Error processing with model: <camel.models.zhipuai_model.ZhipuAIModel object at 0x7db1563c9550>
ERROR:camel.models.model_manager:Error processing with model: <camel.models.zhipuai_model.ZhipuAIModel object at 0x7db1563c9550>
ERROR:camel.camel.agents.chat_agent:Rate limit exhausted after 3 attempts


ModelProcessingError: Unable to process messages: Error code: 429 - {'error': {'code': '1113', 'message': '余额不足或无可用资源包,请充值。'}}